In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import glob

In [ ]:
# Use the glob module to list out the files
glob.glob("Resources/*.xls")

In [ ]:
# The CSV file path
sales_file = "Resources/sales.xls"

# Read the sales data into pandas and remove the header
sales_df = pd.read_excel(sales_file, header=None)
sales_df.head(5)

In [ ]:
# List of all of columns
sales_df.columns

In [ ]:
# Drop the first three rows containing irrelevant information for this analysis by using the index 
sales_row_dropped = sales_df.drop(sales_df.index[[0,1,2]])
sales_row_dropped = sales_row_dropped.reset_index()
sales_row_dropped = sales_row_dropped.drop(['index'], axis=1)
sales_row_dropped.head(5)

In [ ]:
new_header = sales_row_dropped.iloc[0] #grab the first row for the header
sales_new_df = sales_row_dropped[1:] #take the data less the header row
sales_new_df.columns = new_header #set the header row as the df header
sales_new_df.head(5)

In [ ]:
# Stack the dataframe in order to have year in columns and set index to NAICS Code and Kind of Business
sales_stack_df = sales_new_df.set_index(['NAICS Code','Kind of business']).stack()

In [ ]:
# Create a new dataframe with series
sales_stack_new_df = pd.DataFrame(sales_stack_df)

In [ ]:
# Rename index columns
sales_stack_new_df.index.names = ['NAICS Code', 'Kind of Business', 'Year']
   
# Reset the index
df_new_sales = sales_stack_new_df.reset_index()
df_new_sales.rename(columns={0:'Retail Sales Amount'}, inplace=True)

# Drop NAN
df_new_sales = df_new_sales.dropna()

In [ ]:
# Format the year to datetime and remove the decimal
df_new_sales['Year'] = pd.to_datetime(df_new_sales['Year'], format='%Y').dt.strftime('%Y')

In [ ]:
# Format Kind of Business and Retail Sales Amount columns
df_new_sales['Retail Sales Amount'] = df_new_sales['Retail Sales Amount'].astype(int)
#df_new_sales['Kind of Business'] = df_new_sales['Kind of Business'].astype(str)

In [ ]:
# Create new data with select columns
sales_transformed = df_new_sales[['Year', 'Kind of Business', 'Retail Sales Amount']].copy()
sales_transformed.head(5)

In [ ]:
sales_transformed.dtypes

In [ ]:
# Reset index after dropping NAN and reformatting Year and Retail Sales Amount data type
sales_transformed = sales_transformed.reset_index(drop=True)

In [ ]:
# Pivot the table to set Year as index
sales_transformed = pd.pivot_table(sales_transformed, index=['Year'], values=['Retail Sales Amount'])
sales_transformed['Retail Sales Amount'] = sales_transformed['Retail Sales Amount'].astype(int)
sales_transformed.head(5)

In [ ]:
# Function to loop through the resources folder and create dataframe for other excel files 

all_df = []
for f in glob.glob("Resources/*.xls"):
        df = pd.read_excel(f, header=None)
        df_row_dropped = df.drop(df.index[[0,1,2]])
        df_row_dropped = df_row_dropped.reset_index()
        df_row_dropped = df_row_dropped.drop(['index'], axis=1)
        new_header = df_row_dropped.iloc[0]
        df_new = df_row_dropped[1:]
        df_new.columns = new_header
        df_new = df_new.set_index(['NAICS Code','Kind of business']).stack()
        df_new = pd.DataFrame(df_new)
        df_new.index.names = ['NAICS Code', 'Kind of Business', 'Year']
        df_final = df_new.reset_index()
        df_final.rename(columns={0: f}, inplace=True)
        df_final = df_final.dropna()
        df_final['Year'] = pd.to_datetime(df_final['Year'], format='%Y').dt.strftime('%Y')
        df_final[f] = df_final[f].astype(int)
        df_transformed = df_final[['Year', 'Kind of Business', f]].copy()
        df_transformed = df_transformed.reset_index(drop=True)
        df_transformed = pd.pivot_table(df_transformed, index=['Year'], values=[f])
        df_transformed[f] = df_transformed[f].astype(int)
        
        #df_final.reset_index(drop=True)
        #df_final = pd.pivot_table(df_final, index=['Year'], values=[f])
        all_df.append(df_transformed)
        print(f)

In [ ]:
# Rename the dataframe by accessing to the list by index
gm_df = all_df[0]
inventories_df = all_df[1]
purchases_df = all_df[2]

In [ ]:

gm_df.head(2)

In [ ]:
# Rename columns and format data types
gm_df.rename(columns={'Resources\gross_margin.xls':'Gross_Margin_Amount'}, inplace=True)
inventories_df.rename(columns={'Resources\inventories.xls':'Inventories_Amount'}, inplace=True)
purchases_df.rename(columns={'Resources\purchases.xls':'Purchases_Amount'}, inplace=True)

In [ ]:
# Create database connection
connection_string = ""
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
# Load DataFrames into database
sales_transformed.head()

In [ ]:
sales_transformed.to_sql(name='sales', con=engine, if_exists='append', index=True)

In [ ]:
purchases_df.to_sql(name='purchases', con=engine, if_exists='append', index=True)

In [ ]:
inventories_df.to_sql(name='inventories', con=engine, if_exists='append', index=True)

In [ ]:
gm_df.to_sql(name='grossmargin', con=engine, if_exists='append', index=True)

In [ ]:
# Confirm data has been added by quering the sales
pd.read_sql_query('select * from sales', con=engine).head()